<a href="https://colab.research.google.com/github/Abdodusoky/Company/blob/main/Nasa_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# space_bio_knowledge_engine_app.py
"""
Space Biology Knowledge Engine — Streamlit app with CLI fallback

This file fixes environments where `streamlit` (or other optional libs) are not installed.
Behavior:
- If `streamlit` is available, the full interactive Streamlit app will be used (same functionality as before).
- If `streamlit` is NOT available, the script falls back to a command-line interface (CLI) that:
  - Loads the dataset (default: /mnt/data/SB_publication_PMC.csv)
  - Builds topics (TF-IDF + NMF)
  - Builds embeddings (SentenceTransformers if available; otherwise TF-IDF vectors)
  - Allows running a search query from the CLI and saves outputs to disk

How to run (Streamlit UI, if streamlit installed):
    streamlit run space_bio_knowledge_engine_app.py

How to run (CLI fallback -- works without streamlit):
    python space_bio_knowledge_engine_app.py --help

This file intentionally avoids hard crashes when optional packages are missing and provides helpful console output.
"""

from __future__ import annotations
import os
import sys
import argparse
import textwrap
import re
from typing import List, Tuple, Optional

# Optional UI: streamlit
try:
    import streamlit as st
    STREAMLIT_AVAILABLE = True
except Exception:
    st = None
    STREAMLIT_AVAILABLE = False

# Standard data science libs
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity
import plotly.express as px

# Optional heavy libs
try:
    from sentence_transformers import SentenceTransformer
    SSBERT_AVAILABLE = True
except Exception:
    SentenceTransformer = None
    SSBERT_AVAILABLE = False

try:
    import nltk
    nltk.download('punkt', quiet=True)
    NLTK_AVAILABLE = True
except Exception:
    NLTK_AVAILABLE = False

# optional OpenAI & transformers
try:
    import openai
except Exception:
    openai = None

try:
    from transformers import pipeline
except Exception:
    pipeline = None

# Provide caching decorator fallbacks when streamlit is not available
if STREAMLIT_AVAILABLE:
    cache_resource = st.cache_resource
    cache_data = st.cache_data
else:
    def cache_resource(f):
        return f
    def cache_data(f):
        return f

# ---------------------------
# Configuration / Constants
# ---------------------------
DATA_PATH_DEFAULT = os.environ.get('SB_DATA_PATH', 'SB_publication_PMC.csv')
EMBEDDING_MODEL_NAME = 'all-mpnet-base-v2'
NUM_TOPICS = 8
TOP_K = 5

# ---------------------------
# Utility classes & functions
# ---------------------------
class EmbeddingModelWrapper:
    """Wrapper that uses SentenceTransformers if available, otherwise TF-IDF vectors as a fallback.

    The encode(...) method returns a numpy array suitable for cosine similarity.
    """
    def __init__(self, model_name: str = EMBEDDING_MODEL_NAME, tfidf_max_features: int = 10000):
        self.kind = 'sbert' if SSBERT_AVAILABLE else 'tfidf'
        self.model_name = model_name
        if self.kind == 'sbert':
            self.model = SentenceTransformer(model_name)
            self.tfidf_vectorizer = None
        else:
            self.model = None
            self.tfidf_vectorizer = TfidfVectorizer(max_features=tfidf_max_features, stop_words='english')

    def fit_transform_for_fallback(self, texts: List[str]) -> np.ndarray:
        # Fit TF-IDF on texts and return dense array
        X = self.tfidf_vectorizer.fit_transform(texts)
        return X.toarray()

    def encode(self, texts: List[str], convert_to_numpy: bool = True, show_progress_bar: bool = False) -> np.ndarray:
        if self.kind == 'sbert':
            return self.model.encode(texts, convert_to_numpy=convert_to_numpy, show_progress_bar=show_progress_bar)
        else:
            # If our vectorizer hasn't been fit yet, fit it on provided texts (caller usually passes full corpus)
            try:
                X = self.tfidf_vectorizer.transform(texts)
            except Exception:
                X = self.tfidf_vectorizer.fit_transform(texts)
            return X.toarray()


def load_data(path: str = DATA_PATH_DEFAULT, nrows: Optional[int] = None) -> pd.DataFrame:
    if not os.path.exists(path):
        raise FileNotFoundError(f"Dataset not found at {path}. Place the CSV there or provide --data-path.")
    df = pd.read_csv(path, nrows=nrows)
    return df


def preprocess_row(row: pd.Series) -> str:
    parts = []
    for col in ['title', 'abstract', 'body', 'authors', 'journal', 'year']:
        if col in row and pd.notnull(row[col]):
            parts.append(str(row[col]))
    text = '\n'.join(parts)
    text = re.sub(r"\s+", ' ', text)
    return text


def prepare_corpus(df: pd.DataFrame, text_col: str = 'combined_text') -> List[str]:
    if text_col not in df.columns:
        df[text_col] = df.apply(preprocess_row, axis=1)
    texts = df[text_col].fillna('').astype(str).tolist()
    return texts

# ---------------------------
# Embeddings + Topics
# ---------------------------
@cache_resource
def get_embedding_model() -> EmbeddingModelWrapper:
    return EmbeddingModelWrapper()

@cache_resource
def build_embeddings(texts: List[str], model_wrapper: EmbeddingModelWrapper) -> np.ndarray:
    if model_wrapper.kind == 'tfidf':
        # Fit TF-IDF on the entire corpus to get consistent vectors
        emb = model_wrapper.fit_transform_for_fallback(texts)
    else:
        emb = model_wrapper.encode(texts, convert_to_numpy=True, show_progress_bar=True)
    # normalize rows to unit vectors to use cosine similarity with dot product
    norms = np.linalg.norm(emb, axis=1, keepdims=True)
    norms[norms == 0] = 1.0
    emb = emb / norms
    return emb

@cache_data
def make_topics(texts: List[str], n_topics: int = NUM_TOPICS, n_features: int = 5000) -> Tuple[List[Tuple[int, List[str]]], np.ndarray]:
    tfidf = TfidfVectorizer(max_features=n_features, stop_words='english')
    X = tfidf.fit_transform(texts)
    nmf = NMF(n_components=n_topics, random_state=0)
    W = nmf.fit_transform(X)
    H = nmf.components_
    feature_names = tfidf.get_feature_names_out()
    topics = []
    for topic_idx, topic in enumerate(H):
        top_features = [feature_names[i] for i in topic.argsort()[:-11:-1]]
        topics.append((topic_idx, top_features))
    return topics, W

# ---------------------------
# Summarization & generation helpers
# ---------------------------
def call_openai_completion(prompt: str, max_tokens: int = 256, temperature: float = 0.2) -> Optional[str]:
    key = os.environ.get('OPENAI_API_KEY')
    if not key or openai is None:
        return None
    openai.api_key = key
    try:
        # Use a generic ChatCompletion call if available
        resp = openai.ChatCompletion.create(
            model='gpt-4o' if getattr(openai, 'ChatCompletion', None) else 'gpt-3.5-turbo',
            messages=[{'role':'user','content':prompt}],
            max_tokens=max_tokens,
            temperature=temperature,
        )
        return resp['choices'][0]['message']['content']
    except Exception as e:
        print(f"OpenAI call failed: {e}")
        return None


def local_summarize(text: str, max_length: int = 150) -> Optional[str]:
    if pipeline is None:
        return None
    try:
        summarizer = pipeline('summarization', model='facebook/bart-large-cnn', truncation=True)
        out = summarizer(text, max_length=max_length, min_length=30, do_sample=False)
        return out[0]['summary_text']
    except Exception as e:
        print(f"Local summarization failed: {e}")
        return None

# ---------------------------
# Search & extractive answer
# ---------------------------

def search_texts(query: str, texts: List[str], embeddings: np.ndarray, model_wrapper: EmbeddingModelWrapper, top_k: int = TOP_K) -> List[Tuple[int, float]]:
    q_emb = model_wrapper.encode([query], convert_to_numpy=True)
    # normalize
    q_emb = q_emb / (np.linalg.norm(q_emb, axis=1, keepdims=True) + 1e-12)
    sims = cosine_similarity(q_emb, embeddings)[0]
    top_idx = sims.argsort()[::-1][:top_k]
    return [(int(i), float(sims[i])) for i in top_idx]


def extractive_answer(user_question: str, texts: List[str], embeddings: np.ndarray, model_wrapper: EmbeddingModelWrapper, top_k: int = TOP_K) -> Tuple[List[str], np.ndarray]:
    hits = search_texts(user_question, texts, embeddings, model_wrapper, top_k=top_k)
    indices = [i for i, _ in hits]
    scores = np.array([s for _, s in hits])
    answers = [texts[i] for i in indices]
    return answers, scores

# ---------------------------
# Streamlit UI (if available)
# ---------------------------

def main_streamlit(data_path: str = DATA_PATH_DEFAULT):
    # This function assumes streamlit is available.
    st.set_page_config(layout='wide', page_title='Space Biology Knowledge Engine')
    st.title('🚀 Space Biology Knowledge Engine — Research explorer & chatbot')

    st.sidebar.header('Configuration')
    use_sample = st.sidebar.checkbox('Load sample subset (fast)', value=True)
    nrows = 200 if use_sample else None

    if not os.path.exists(data_path):
        st.error(f"Dataset not found at {data_path}. Please upload or place it there.")
        return

    df = load_data(data_path, nrows=nrows)

    if 'combined_text' not in df.columns:
        st.info('Combining columns into `combined_text` (title, abstract, body, authors)')
        df['combined_text'] = df.apply(preprocess_row, axis=1)

    with st.expander('Dataset preview'):
        st.dataframe(df.head(10))

    if st.sidebar.button('Build topics'):
        with st.spinner('Building topics (TF-IDF + NMF)...'):
            texts = prepare_corpus(df)
            topics, W = make_topics(texts)
            st.subheader('Topics (TF-IDF + NMF)')
            for tid, words in topics:
                st.markdown(f"**Topic {tid}**: {', '.join(words)}")

    if st.sidebar.button('Build embeddings'):
        with st.spinner('Loading embedding model and encoding texts...'):
            model_wrapper = get_embedding_model()
            texts = prepare_corpus(df)
            embeddings = build_embeddings(texts, model_wrapper)
            st.session_state['embeddings'] = embeddings
            st.session_state['model_wrapper'] = model_wrapper
            st.success('Embeddings built and cached')

    # Search
    st.header('🔎 Search & Explore')
    query = st.text_input('Enter a search question or keywords')
    k = st.slider('Results to show', min_value=1, max_value=20, value=5)

    if st.button('Search') and query.strip():
        if 'embeddings' not in st.session_state:
            st.warning('Please build embeddings first (sidebar).')
        else:
            model_wrapper = st.session_state['model_wrapper']
            embeddings = st.session_state['embeddings']
            texts = prepare_corpus(df)
            hits = search_texts(query, texts, embeddings, model_wrapper, top_k=k)
            st.write(f'Found top {len(hits)} documents:')
            for rank, (idx, score) in enumerate(hits, start=1):
                row = df.iloc[idx]
                st.markdown(f"**{rank}. {row.get('title','(no title)')}** — score {score:.3f}")
                if 'abstract' in row and pd.notnull(row['abstract']):
                    st.write(row['abstract'][:800] + ('...' if len(str(row['abstract']))>800 else ''))

    # Chatbot (RAG) - simplified
    st.header('🤖 Research Chatbot (RAG)')
    user_q = st.text_area('Ask a research question about space biology (use the dataset)')
    bot_mode = st.radio('Answering mode', ['Retrieval + Extractive', 'Retrieval + Generative (OpenAI)', 'Local Generative (HF)'])

    if st.button('Ask') and user_q.strip():
        if 'embeddings' not in st.session_state:
            st.warning('Please build embeddings first (sidebar).')
        else:
            model_wrapper = st.session_state['model_wrapper']
            embeddings = st.session_state['embeddings']
            texts = prepare_corpus(df)
            answers, scores = extractive_answer(user_q, texts, embeddings, model_wrapper, top_k=TOP_K)
            st.subheader('Retrieved (extractive)')
            for i, (a, s) in enumerate(zip(answers, scores), start=1):
                st.markdown(f"**Snippet {i} (score {s:.3f})**")
                st.write(textwrap.shorten(a, width=600, placeholder='...'))

            if bot_mode == 'Retrieval + Generative (OpenAI)':
                prompt = 'You are an expert assistant for space biology research. Use the following retrieved documents as evidence, then answer the user question succinctly.\n\n'
                for j, doc in enumerate(answers, start=1):
                    prompt += f"[DOC {j}] " + doc[:1000] + '\n\n'
                prompt += f"User question: {user_q}\n\nAnswer concisely, cite which DOC number you used."
                out = call_openai_completion(prompt)
                if out:
                    st.subheader('Generative answer (OpenAI)')
                    st.write(out)
                else:
                    st.warning('OpenAI not available or call failed. Consider extractive mode.')

            if bot_mode == 'Local Generative (HF)':
                if pipeline is None:
                    st.warning('Local transformers pipeline not available.')
                else:
                    gen_pipe = pipeline('text-generation', model='gpt2', device=-1)
                    context = '\n\n'.join(answers[:3])
                    prompt = f"Context: {context}\n\nQuestion: {user_q}\nAnswer:"
                    res = gen_pipe(prompt, max_length=256, do_sample=False)
                    st.subheader('Local generative answer')
                    st.write(res[0]['generated_text'])

    # Visualization (Streamlit only)
    st.header('📊 Interactive Research Visualizations')
    viz_choice = st.selectbox('Choose visualization', ['Publications per year', 'Top authors', 'Topic word list'])

    if viz_choice == 'Publications per year':
        if 'year' in df.columns:
            df_year = df.dropna(subset=['year'])
            try:
                df_year['year'] = df_year['year'].astype(int)
                counts = df_year.groupby('year').size().reset_index(name='count')
                fig = px.bar(counts, x='year', y='count', title='Publications per year')
                st.plotly_chart(fig, use_container_width=True)
            except Exception:
                st.info('Year column found but could not be converted to int.')
        else:
            st.info('No `year` column found in dataset')

    if viz_choice == 'Top authors':
        if 'authors' in df.columns:
            all_auth = df['authors'].dropna().astype(str).str.split('[;,]').explode().str.strip()
            top = all_auth.value_counts().head(20).reset_index()
            top.columns = ['author','count']
            fig = px.bar(top, x='author', y='count', title='Top authors', labels={'author':'Author', 'count':'Papers'})
            st.plotly_chart(fig, use_container_width=True)
        else:
            st.info('No `authors` column found in dataset')

    if viz_choice == 'Topic word list':
        texts = prepare_corpus(df)
        topics, W = make_topics(texts)
        cols = st.columns(2)
        for i, (tid, words) in enumerate(topics):
            with cols[i % 2]:
                st.markdown(f"### Topic {tid}")
                st.write(', '.join(words))

    # Export
    st.header('📁 Export')
    st.write('Enter comma-separated indices to export (e.g. 0,5,10)')
    indices_text = st.text_input('Indices to export')
    if st.button('Export selected'):
        try:
            idxs = [int(x.strip()) for x in indices_text.split(',') if x.strip() != '']
            sub = df.iloc[idxs]
            csv = sub.to_csv(index=False).encode('utf-8')
            st.download_button('Download CSV', data=csv, file_name='selected_papers.csv', mime='text/csv')
        except Exception as e:
            st.error(f'Failed to export: {e}')

    st.sidebar.markdown('---')
    st.sidebar.write('Tips: Build embeddings once, then use search/chat. For high-quality generation, set OPENAI_API_KEY.')

# ---------------------------
# CLI fallback
# ---------------------------

def print_table(df: pd.DataFrame, n: int = 5):
    print(df.head(n).to_string(index=False))


def main_cli(args: argparse.Namespace):
    data_path = args.data_path or DATA_PATH_DEFAULT
    nrows = args.nrows
    try:
        df = load_data(data_path, nrows=nrows)
    except FileNotFoundError as e:
        print(e)
        sys.exit(1)

    if 'combined_text' not in df.columns:
        df['combined_text'] = df.apply(preprocess_row, axis=1)

    texts = prepare_corpus(df)
    print(f"Loaded {len(df)} rows from {data_path}")

    # Build topics
    print('\nBuilding topics (TF-IDF + NMF)...')
    topics, W = make_topics(texts, n_topics=args.num_topics)
    print('\nTop topics:')
    for tid, words in topics:
        print(f"Topic {tid}: {', '.join(words)}")

    # Build embeddings
    print('\nBuilding embeddings (SentenceTransformers if available, otherwise TF-IDF fallback)...')
    model_wrapper = get_embedding_model()
    embeddings = build_embeddings(texts, model_wrapper)
    print(f"Embeddings shape: {embeddings.shape} (model kind={model_wrapper.kind})")

    # Save embeddings optionally
    if args.save_embeddings:
        np.save(args.save_embeddings, embeddings)
        print(f"Saved embeddings to {args.save_embeddings}")

    # Quick search if query provided
    if args.query:
        hits = search_texts(args.query, texts, embeddings, model_wrapper, top_k=args.top_k)
        print(f"\nTop {len(hits)} results for query: '{args.query}'")
        for rank, (idx, score) in enumerate(hits, start=1):
            row = df.iloc[idx]
            title = row.get('title', '(no title)')
            abstract = row.get('abstract', '')
            print(f"{rank}. index={idx} score={score:.4f} title={title}")
            if pd.notnull(abstract):
                print(textwrap.fill(str(abstract)[:400], width=120))
                print('---')

    # Run a simple smoke test if requested
    if args.smoke_test:
        sample_q = args.query or 'microgravity'
        print(f"\nRunning smoke test with query '{sample_q}'")
        hits = search_texts(sample_q, texts, embeddings, model_wrapper, top_k=3)
        print('Smoke test results:')
        for i, (idx, score) in enumerate(hits, start=1):
            print(f"{i}. idx={idx} score={score:.4f} title={df.iloc[idx].get('title','(no title)')}")

    print('\nCLI run finished.')

# ---------------------------
# Argument parsing & entry point
# ---------------------------

def make_argparser() -> argparse.ArgumentParser:
    p = argparse.ArgumentParser(description='Space Biology Knowledge Engine (Streamlit UI if available; CLI fallback otherwise)')
    p.add_argument('--data-path', type=str, default=DATA_PATH_DEFAULT, help='Path to CSV dataset')
    p.add_argument('--nrows', type=int, default=None, help='Read only first N rows (for quick tests)')
    p.add_argument('--num-topics', type=int, default=NUM_TOPICS, help='Number of NMF topics')
    p.add_argument('--query', type=str, default=None, help='Run a quick search query (CLI mode)')
    p.add_argument('--top-k', type=int, default=TOP_K, help='Number of top results to show')
    p.add_argument('--save-embeddings', type=str, default=None, help='Save embeddings to .npy file')
    p.add_argument('--smoke-test', action='store_true', help='Run a small smoke test after building embeddings')
    return p


if __name__ == '__main__':
    parser = make_argparser()
    args, unknown = parser.parse_known_args()

    if STREAMLIT_AVAILABLE:
        # If streamlit available and script is executed with `streamlit run`, run the UI function.
        # When using `python script.py` in an environment with streamlit, we will still launch CLI by default.
        invoked_via_streamlit = 'STREAMLIT_SERVER_RUN' in os.environ or 'streamlit' in sys.argv[0]
        if invoked_via_streamlit:
            # Streamlit expects to execute top-level code; define a wrapper
            main_streamlit(data_path=args.data_path)
        else:
            # User ran `python script.py` in environment with streamlit; provide CLI as default
            main_cli(args)
    else:
        # No streamlit — run CLI fallback
        main_cli(args)

# ---------------------------
# End of file
# ---------------------------


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2025-09-17 12:47:43.569 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-09-17 12:47:43.573 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-17 12:47:43.576 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-17 12:47:43.826 
  command:

    streamlit run /usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-09-17 12:47:43.827 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-17 12:47:43.830 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.